AutoGluon - Predicción de ventas (tn) por producto para febrero 2020

In [1]:
# 📦 1. Importar librerías
import os
import pandas as pd
from datetime import datetime
import time
import json
import re

In [2]:
# 💬 Instalar AutoGluon y kaggle si es necesario
%pip install autogluon.timeseries
%pip install kaggle

from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame
from autogluon.common import space as ag_space

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


C:\Proyectos\EM\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 📄 2. Cargar datasets
df_sellin = pd.read_csv("sell-in.txt", sep="\t")
df_productos = pd.read_csv("tb_productos.txt", sep="\t")

In [4]:
# 📄 Leer lista de productos a predecir
with open("productos_pred.txt", "r") as f:
    product_ids = [int(line.strip()) for line in f if line.strip().isdigit()]

In [5]:
# 🧹 3. Preprocesamiento
# Convertir periodo a datetime
df_sellin['timestamp'] = pd.to_datetime(df_sellin['periodo'], format='%Y%m')

In [6]:
# Filtrar hasta dic 2019 y productos requeridos
df_filtered = df_sellin[
    (df_sellin['timestamp'] <= '2019-12-01') &
    (df_sellin['product_id'].isin(product_ids))
]

In [7]:
# Agregar tn por periodo, cliente y producto
df_grouped = df_filtered.groupby(['timestamp', 'customer_id', 'product_id'], as_index=False)['tn'].sum()

In [8]:
# Agregar tn total por periodo y producto
df_monthly_product = df_grouped.groupby(['timestamp', 'product_id'], as_index=False)['tn'].sum()

In [9]:
# Agregar columna 'item_id' para AutoGluon
df_monthly_product['item_id'] = df_monthly_product['product_id']

In [10]:
# ⏰ 4. Crear TimeSeriesDataFrame
ts_data = TimeSeriesDataFrame.from_data_frame(
    df_monthly_product,
    id_column='item_id',
    timestamp_column='timestamp'
)

In [11]:
# Completar valores faltantes
ts_data = ts_data.fill_missing_values()

# Convertir columnas numéricas float64 a float32
def convert_float64_to_float32(df):
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = df[col].astype('float32')
    return df

# Aplicar conversión a los dataframes relevantes
df_sellin = convert_float64_to_float32(df_sellin)
df_productos = convert_float64_to_float32(df_productos)
df_grouped = convert_float64_to_float32(df_grouped) if 'df_grouped' in locals() else None
df_monthly_product = convert_float64_to_float32(df_monthly_product) if 'df_monthly_product' in locals() else None

# 🔍 Validación de datos antes del entrenamiento
print("🔍 Validando datos de entrada...")
print(f"📊 Forma de los datos: {ts_data.shape}")
print(f"📅 Rango de fechas: {ts_data.index.get_level_values('timestamp').min()} - {ts_data.index.get_level_values('timestamp').max()}")
print(f"🏷️ Número de productos únicos: {ts_data.index.get_level_values('item_id').nunique()}")
print(f"📈 Valores nulos en target: {ts_data['tn'].isnull().sum()}")

# Verificar que tenemos suficientes datos
if ts_data.empty:
    raise ValueError("❌ Error: Los datos están vacíos")
if ts_data['tn'].isnull().all():
    raise ValueError("❌ Error: Todos los valores target son nulos")
if ts_data.index.get_level_values('item_id').nunique() == 0:
    raise ValueError("❌ Error: No hay productos en los datos")

print("✅ Validación de datos completada exitosamente")

🔍 Validando datos de entrada...
📊 Forma de los datos: (22349, 2)
📅 Rango de fechas: 2017-01-01 00:00:00 - 2019-12-01 00:00:00
🏷️ Número de productos únicos: 780
📈 Valores nulos en target: 0
✅ Validación de datos completada exitosamente


In [12]:
# 🔧 5. Funciones auxiliares para el bucle iterativo
def get_kaggle_score():
    """Obtiene el score más reciente de Kaggle"""
    try:
        output = os.popen('kaggle competitions submissions -c labo-iii-edicion-2025-v').read()
        lines = output.strip().split('\n')
        if len(lines) > 2:
            # La segunda línea contiene la submission más reciente
            recent_line = lines[2]
            print(lines[2])
            # Extraer el score con un patrón más robusto
            # Buscar números decimales que podrían ser el score
            score_matches = re.findall(r'(\d+\.\d+)', recent_line)
            if score_matches:
                # Tomar el último número decimal encontrado (generalmente es el score)
                return float(score_matches[-1])
            # Si no hay decimales, buscar números enteros
            int_matches = re.findall(r'\b(\d+)\b', recent_line)
            if int_matches:
                return float(int_matches[-1])
    except Exception as e:
        print(f"⚠️ Error obteniendo score de Kaggle: {e}")
    return None

In [13]:
def submit_to_kaggle(resultado, iteration):
    """Sube predicción a Kaggle y retorna el score"""
    timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_filename = f"predicciones_iter_{iteration}_{timestamp_str}.csv"
    resultado.to_csv(output_filename, index=False)

    message = f"Iteración {iteration} - Mejora automática {timestamp_str}"
    os.system(f'kaggle competitions submit -c labo-iii-edicion-2025-v -f {output_filename} -m "{message}"')

    # Esperar más tiempo para que Kaggle procese
    print(f"⏳ Esperando 3 minutos para que Kaggle procese la submission...")
    time.sleep(180)  # Incrementar a 3 minutos
    return get_kaggle_score()

In [14]:
def get_improved_hyperparameters(iteration, best_score, current_score):
    """Ajusta hiperparámetros basándose en el rendimiento"""
    import random

    # Print información de estado actual
    print(f"🔧 Ajustando hiperparámetros para iteración {iteration}")
    print(f"📊 Score actual: {current_score if current_score else 'N/A'}")
    print(f"🏆 Mejor score hasta ahora: {best_score if best_score != float('inf') else 'N/A'}")

    # Determinar si vamos mejorando
    improving = False
    if current_score and best_score != float('inf'):
        improving = current_score < best_score
        improvement_diff = best_score - current_score if improving else current_score - best_score
        if improving:
            print(f"📈 Tendencia: MEJORANDO (diferencia: {improvement_diff:.6f}) - Explorando modelos más complejos")
        else:
            print(f"📉 Tendencia: SIN MEJORAR (diferencia: {improvement_diff:.6f}) - Ajustando estrategia")
    else:
        print("🎯 Primera iteración - Explorando configuración base")

    # Acceder a variables globales para detectar estancamiento
    global iterations_without_improvement
    current_stagnation = iterations_without_improvement if 'iterations_without_improvement' in globals() else 0

    if current_stagnation > 0:
        print(f"🔄 Detectado estancamiento de {current_stagnation} iteraciones - Aplicando cambios DRÁSTICOS")

    # Usar iteración y estancamiento para cambios MÁS AGRESIVOS
    random.seed(iteration * 123 + current_stagnation * 456)  # Más variación en la semilla

    # RANGOS DRAMÁTICAMENTE EXPANDIDOS - Siempre usar rangos amplios
    epochs_range = [50, 100, 150, 200, 250, 300, 400, 500, 600]  # Épocas mucho más altas
    lr_range = [1e-1, 5e-2, 1e-2, 5e-3, 1e-3, 5e-4, 1e-4, 5e-5, 1e-5, 1e-6]  # Rangos extremos
    hidden_sizes = [64, 128, 256, 512, 768, 1024, 1536, 2048]  # Tamaños más grandes
    layers_range = [2, 3, 4, 5, 6, 7, 8, 10, 12]  # Más capas profundas
    dropout_range = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]  # Rango completo

    # Con estancamiento, usar TODOS los valores disponibles
    if current_stagnation > 3:
        print("💥 MODO EXTREMO: Usando rangos máximos por estancamiento")
        epochs_range.extend([700, 800, 1000])  # Épocas extremas
        hidden_sizes.extend([3072, 4096])  # Tamaños gigantes
        layers_range.extend([15, 20])  # Capas muy profundas
        lr_range.extend([2e-1, 5e-6, 1e-7])  # LR extremos

    # Seleccionar MUCHAS más combinaciones - mínimo 5-8 opciones
    variation_factor = max(3, current_stagnation)
    epoch_choices = random.sample(epochs_range, min(6 + variation_factor, len(epochs_range)))
    lr_choices = random.sample(lr_range, min(6 + variation_factor, len(lr_range)))
    hidden_choices = random.sample(hidden_sizes, min(5 + variation_factor, len(hidden_sizes)))
    layer_choices = random.sample(layers_range, min(5 + variation_factor, len(layers_range)))
    dropout_choices = random.sample(dropout_range, min(5 + variation_factor, len(dropout_range)))

    print(f"⚙️ Epochs seleccionados: {sorted(epoch_choices)}")
    print(f"⚙️ Learning rates: {sorted(lr_choices, reverse=True)}")
    print(f"⚙️ Hidden sizes: {sorted(hidden_choices)}")
    print(f"⚙️ Layers: {sorted(layer_choices)}")
    print(f"⚙️ Dropout rates: {sorted(dropout_choices)}")

    # Configuraciones específicas por modelo con MÁXIMA variación
    deepar_epochs = random.sample([100, 150, 200, 300, 400, 500, 600, 800], min(6, 8))
    tft_epochs = random.sample([50, 100, 150, 200, 300, 400, 500], min(5, 7))
    patch_epochs = random.sample([50, 100, 150, 200, 300, 400, 500], min(5, 7))

    # Variaciones EXTREMAS en patch lengths y strides
    patch_lens = random.sample([4, 8, 12, 16, 24, 32, 48, 64, 96, 128], min(6, 10))
    strides = random.sample([2, 4, 6, 8, 12, 16, 24, 32, 48, 64], min(6, 10))
    attention_heads = random.sample([4, 6, 8, 12, 16, 20, 24, 32], min(5, 8))

    # Configuraciones base SIEMPRE con modelos complejos
    base_hyperparameters = {
        'DeepAR': {
            'epochs': ag_space.Categorical(*deepar_epochs),
            'learning_rate': ag_space.Categorical(*lr_choices),
            'hidden_size': ag_space.Categorical(*hidden_choices),
            'num_layers': ag_space.Categorical(*layer_choices),
            'dropout_rate': ag_space.Categorical(*dropout_choices)
        },
        'TemporalFusionTransformer': {
            'epochs': ag_space.Categorical(*tft_epochs),
            'learning_rate': ag_space.Categorical(*lr_choices),
            'hidden_size': ag_space.Categorical(*hidden_choices),
            'num_attention_heads': ag_space.Categorical(*attention_heads)
        },
        'PatchTST': {
            'epochs': ag_space.Categorical(*patch_epochs),
            'learning_rate': ag_space.Categorical(*lr_choices),
            'patch_len': ag_space.Categorical(*patch_lens),
            'stride': ag_space.Categorical(*strides)
        }
    }

    # SIEMPRE agregar modelos experimentales desde iteración 2
    if iteration >= 2:
        print("🚀 Agregando TODOS los modelos experimentales disponibles")

        # TiDE con configuraciones agresivas
        tide_epochs = random.sample([150, 200, 300, 400, 500, 600, 800], min(5, 7))
        tide_hidden = random.sample([512, 768, 1024, 1536, 2048, 3072], min(5, 6))
        tide_layers = random.sample([4, 6, 8, 10, 12, 15], min(4, 6))

        base_hyperparameters.update({
            'TiDE': {
                'epochs': ag_space.Categorical(*tide_epochs),
                'learning_rate': ag_space.Categorical(*lr_choices),
                'hidden_size': ag_space.Categorical(*tide_hidden),
                'num_layers': ag_space.Categorical(*tide_layers),
                'dropout': ag_space.Categorical(*dropout_choices)
            }
        })

        # Chronos con TODOS los tamaños
        chronos_sizes = ['tiny', 'mini', 'small', 'base', 'large'] if iteration > 5 else ['tiny', 'mini', 'small', 'base']
        base_hyperparameters.update({
            'Chronos': {
                'model_size': ag_space.Categorical(*chronos_sizes)
            }
        })

        # NPTS con contextos variables
        context_lengths = random.sample([24, 36, 48, 60, 72, 96, 120], min(5, 7))
        base_hyperparameters.update({
            'NPTS': {
                'context_length': ag_space.Categorical(*context_lengths)
            }
        })

        print(f"   ├─ TiDE epochs: {sorted(tide_epochs)}")
        print(f"   ├─ TiDE hidden sizes: {sorted(tide_hidden)}")
        print(f"   ├─ Chronos sizes: {chronos_sizes}")
        print(f"   └─ NPTS context lengths: {sorted(context_lengths)}")

    # Modelos baseline con configuraciones mejoradas
    baseline_models = ['AutoETS', 'DynamicOptimizedTheta', 'SeasonalNaive', 'DirectTabular', 'RecursiveTabular']
    base_hyperparameters.update({
        'AutoETS': {},
        'DynamicOptimizedTheta': {},
        'SeasonalNaive': {},
        'DirectTabular': {'ag_args_fit': {'num_gpus': 0, 'verbosity': 2}},
        'RecursiveTabular': {'ag_args_fit': {'num_gpus': 0, 'verbosity': 2}}
    })

    print(f"🎯 Total de modelos configurados: {len(base_hyperparameters)}")
    print(f"📋 Modelos complejos: {[k for k in base_hyperparameters.keys() if k not in baseline_models]}")
    print(f"📋 Modelos baseline: {baseline_models}")
    print(f"🔥 Nivel de AGRESIVIDAD MÁXIMA aplicado: {min(current_stagnation + 5, 10)}/10")
    print("-" * 50)

    return base_hyperparameters

In [ ]:
# 🔄 6. Bucle iterativo de mejora
print("🚀 Iniciando bucle iterativo de mejora de modelo")
print("=" * 60)

# Variables para tracking
best_score = float('inf')
best_iteration = 0
scores_history = []
models_history = []
iterations_without_improvement = 0  # Contador de iteraciones sin mejora
last_improvement_iteration = 0  # Última iteración con mejora

for iteration in range(1, 81):  # 80 iteraciones
    print(f"\n🔄 ITERACIÓN {iteration}/80")
    print("-" * 40)

    try:
        # Crear predictor para esta iteración
        predictor = TimeSeriesPredictor(
            prediction_length=2,
            target='tn',
            freq='MS',
            eval_metric='MASE',
            path=f'AutogluonModels/iteration_{iteration}'
        )

        # PRIMERA ITERACIÓN: Usar configuración por defecto SIN hiperparámetros específicos
        if iteration == 1:
            print("🎯 PRIMERA ITERACIÓN - Entrenamiento con configuración por defecto")
            print("📋 Usando modelos base de AutoGluon sin hiperparámetros específicos")

            # Configuración simple para la primera iteración
            hyperparameters = None  # Usar defaults de AutoGluon
            hyperparameter_tune_kwargs = None  # Sin búsqueda de hiperparámetros

            # Tiempo base más corto para la primera iteración (1 hora)
            adjusted_time = 3600  # 1 hora
            print(f"⏰ Tiempo de entrenamiento inicial: {adjusted_time/3600:.1f} hora")

        else:
            # ITERACIONES 2+: Usar lógica de hiperparámetros existente
            print(f"🔧 ITERACIÓN {iteration} - Optimización avanzada con hiperparámetros")

            # Obtener hiperparámetros mejorados
            current_score = scores_history[-1] if scores_history else None
            hyperparameters = get_improved_hyperparameters(iteration, best_score, current_score)

            # Ajustar trials basándose en las iteraciones sin mejora - MÁS AGRESIVO
            base_trials = min(10 + iteration * 5, 150)  # Incrementar trials base significativamente
            if iterations_without_improvement > 3:
                # Incrementar trials DRAMÁTICAMENTE si no hay mejoras
                trials_multiplier = min(3 + (iterations_without_improvement // 2), 6)
                adjusted_trials = min(base_trials * trials_multiplier, 300)  # Hasta 300 trials
                print(f"⚡ Sin mejoras por {iterations_without_improvement} iteraciones - Incrementando trials a {adjusted_trials}")
            else:
                adjusted_trials = base_trials

            # Configurar búsqueda de hiperparámetros
            hyperparameter_tune_kwargs = {
                'num_trials': adjusted_trials,
                'scheduler': 'local',
                'searcher': 'random'
            }

            print(f"📊 Entrenando con {len(hyperparameters)} modelos...")
            print(f"🔍 Trials de hiperparámetros: {hyperparameter_tune_kwargs['num_trials']}")

            # TIEMPOS DE ENTRENAMIENTO DRAMÁTICAMENTE MÁS LARGOS
            # Tiempo base mucho más alto: mínimo 4 horas, máximo 24 horas
            base_time_hours = 4 + (iteration * 0.5)  # Incrementar 30 min por iteración
            base_time = int(base_time_hours * 3600)  # Convertir a segundos

            # Multiplicadores agresivos por estancamiento
            if iterations_without_improvement > 8:
                time_multiplier = 5.0  # 5x más tiempo si está muy estancado
                adjusted_time = min(int(base_time * time_multiplier), 24 * 3600)  # Máximo 24 horas
                print(f"💥 ESTANCAMIENTO EXTREMO - Tiempo de entrenamiento: {adjusted_time/3600:.1f} horas")
            elif iterations_without_improvement > 5:
                time_multiplier = 3.0  # 3x más tiempo si está estancado
                adjusted_time = min(int(base_time * time_multiplier), 20 * 3600)  # Máximo 20 horas
                print(f"🚨 ESTANCAMIENTO PROLONGADO - Tiempo de entrenamiento: {adjusted_time/3600:.1f} horas")
            elif iterations_without_improvement > 2:
                time_multiplier = 2.0  # 2x más tiempo si no mejora
                adjusted_time = min(int(base_time * time_multiplier), 16 * 3600)  # Máximo 16 horas
                print(f"⚡ Sin mejoras detectadas - Tiempo de entrenamiento: {adjusted_time/3600:.1f} horas")
            else:
                adjusted_time = base_time
                print(f"⏰ Tiempo de entrenamiento base: {adjusted_time/3600:.1f} horas")

        print(f"🕐 Tiempo estimado de finalización: {(datetime.now() + pd.Timedelta(seconds=adjusted_time)).strftime('%H:%M:%S')}")
        print(f"📅 Fecha estimada de finalización: {(datetime.now() + pd.Timedelta(seconds=adjusted_time)).strftime('%Y-%m-%d')}")

        # Entrenar modelo con configuración específica por iteración
        start_time = time.time()

        if iteration == 1:
            # Primera iteración: entrenamiento simple
            predictor.fit(
                train_data=ts_data,
                time_limit=adjusted_time
            )
        else:
            # Iteraciones 2+: entrenamiento con hiperparámetros
            predictor.fit(
                train_data=ts_data,
                hyperparameters=hyperparameters,
                hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
                time_limit=adjusted_time
            )

        training_time = time.time() - start_time

        print(f"⏱️ Tiempo de entrenamiento REAL: {training_time/3600:.2f} horas ({training_time/60:.1f} minutos)")

        # Comparar tiempo real vs planificado
        time_efficiency = (training_time / adjusted_time) * 100
        print(f"📊 Eficiencia de tiempo: {time_efficiency:.1f}% del tiempo asignado")

        if time_efficiency < 50:
            print("⚠️ El entrenamiento terminó mucho antes del tiempo asignado - considera modelos más complejos")
        elif time_efficiency > 95:
            print("✅ Uso completo del tiempo asignado - entrenamiento exhaustivo")

        # Generar predicción
        print("🔮 Generando predicciones...")
        forecast = predictor.predict(ts_data)

        # Verificar que forecast contiene datos válidos
        if forecast is None or 'mean' not in forecast:
            print("❌ Error: No se pudo generar predicción válida")
            scores_history.append(None)
            models_history.append({
                'iteration': iteration,
                'score': None,
                'training_time': training_time,
                'best_model': 'Error',
                'error': 'No prediction generated'
            })
            continue

        # Procesar resultados
        resultado = forecast['mean'].reset_index()

        # Verificar que tenemos datos para febrero 2020
        feb_2020_data = resultado[resultado['timestamp'] == '2020-02-01']
        if feb_2020_data.empty:
            print("⚠️ Advertencia: No hay predicciones para febrero 2020, usando primer mes disponible")
            # Usar el primer mes disponible
            unique_timestamps = resultado['timestamp'].unique()
            if len(unique_timestamps) > 0:
                feb_2020_data = resultado[resultado['timestamp'] == unique_timestamps[0]]

        if feb_2020_data.empty:
            print("❌ Error: No hay predicciones disponibles")
            scores_history.append(None)
            continue

        resultado = feb_2020_data[['item_id', 'mean']].copy()
        resultado.columns = ['product_id', 'tn']

        # Validar que tenemos predicciones válidas
        if resultado.empty or resultado['tn'].isna().all():
            print("❌ Error: Predicciones vacías o inválidas")
            scores_history.append(None)
            continue

        print(f"📈 Productos predichos: {len(resultado)}")
        print(f"📊 Rango de predicciones: {resultado['tn'].min():.2f} - {resultado['tn'].max():.2f}")

        # Subir a Kaggle y obtener score
        print("⬆️ Subiendo a Kaggle...")
        kaggle_score = submit_to_kaggle(resultado, iteration)

        if kaggle_score:
            scores_history.append(kaggle_score)
            print(f"🎯 Score obtenido: {kaggle_score}")

            # Verificar si es el mejor score
            if kaggle_score < best_score:
                best_score = kaggle_score
                best_iteration = iteration
                last_improvement_iteration = iteration
                iterations_without_improvement = 0  # Resetear contador
                print(f"🏆 ¡NUEVO MEJOR SCORE! Mejora: {best_score}")
                print(f"✨ Reseteando contador de iteraciones sin mejora")
            else:
                iterations_without_improvement += 1
                print(f"📉 No mejoró. Mejor score sigue siendo: {best_score} (iteración {best_iteration})")
                print(f"⏳ Iteraciones sin mejora: {iterations_without_improvement}")

                # Estrategias adicionales cuando no hay mejora
                if iterations_without_improvement >= 3:
                    print(f"🔧 Activando estrategias de recuperación (sin mejora por {iterations_without_improvement} iteraciones)")

                if iterations_without_improvement >= 5:
                    print("🚨 Considerando cambios drásticos en próximas iteraciones")

                if iterations_without_improvement >= 10:
                    print("💥 Implementando búsqueda exhaustiva - esto puede tomar más tiempo")
        else:
            print("⚠️ No se pudo obtener score de Kaggle")
            scores_history.append(None)
            iterations_without_improvement += 1

        # Guardar información del modelo
        try:
            model_info = predictor.leaderboard()
            best_model = model_info.iloc[0]['model'] if not model_info.empty else 'Unknown'
            leaderboard_records = model_info.to_dict('records') if not model_info.empty else []
        except Exception as e:
            print(f"⚠️ Error obteniendo leaderboard: {e}")
            best_model = 'Unknown'
            leaderboard_records = []

        models_history.append({
            'iteration': iteration,
            'score': kaggle_score,
            'training_time': training_time,
            'best_model': best_model,
            'leaderboard': leaderboard_records
        })

        print(f"🥇 Mejor modelo local: {best_model}")

        # Guardar progreso
        progress_data = {
            'scores_history': scores_history,
            'models_history': models_history,
            'best_score': best_score,
            'best_iteration': best_iteration,
            'last_updated': datetime.now().isoformat()
        }

        try:
            with open(f'iteration_progress_{datetime.now().strftime("%Y%m%d")}.json', 'w') as f:
                json.dump(progress_data, f, indent=2, default=str)
            print(f"💾 Progreso guardado exitosamente")
        except Exception as e:
            print(f"⚠️ Error guardando progreso: {e}")

    except Exception as e:
        print(f"❌ Error en iteración {iteration}: {e}")
        scores_history.append(None)
        models_history.append({
            'iteration': iteration,
            'score': None,
            'training_time': 0,
            'best_model': 'Error',
            'error': str(e)
        })
        continue

🚀 Iniciando bucle iterativo de mejora de modelo

🔄 ITERACIÓN 1/80
----------------------------------------


Beginning AutoGluon training... Time limit = 3600s
AutoGluon will save models to 'G:\Mi unidad\Maestría\Data Science\Laboratorio de Implementación III\Proyectos\labo3-2025v\AutogluonModels\iteration_1'


🎯 PRIMERA ITERACIÓN - Entrenamiento con configuración por defecto
📋 Usando modelos base de AutoGluon sin hiperparámetros específicos
⏰ Tiempo de entrenamiento inicial: 1.0 hora
🕐 Tiempo estimado de finalización: 17:29:44
📅 Fecha estimada de finalización: 2025-07-20


=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.12.6
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          12
GPU Count:          0
Memory Avail:       4.91 GB / 15.71 GB (31.2%)
Disk Space Avail:   20.20 GB / 100.00 GB (20.2%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': MASE,
 'freq': 'MS',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 2,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'tn',
 'time_limit': 3600,
 'verbosity': 2}

train_data with frequency 'IRREG' has been resampled to frequency 'MS'.
Provided train_data has 22375 rows (NaN fraction=0.1%), 780 time series. Median time series length is 36 (min=4, max=36). 
	Removing 46 short time series from train_data. O

In [ ]:
# 📊 7. Resumen final
print("\n" + "=" * 60)
print("📊 RESUMEN FINAL DE MEJORAS")
print("=" * 60)

for i, score in enumerate(scores_history, 1):
    if score:
        status = "🏆 MEJOR" if i == best_iteration else ""
        print(f"Iteración {i}: {score} {status}")
    else:
        print(f"Iteración {i}: Sin score")

if best_iteration > 0:
    improvement = scores_history[0] - best_score if scores_history[0] else 0
    print(f"\n🎯 Mejor resultado: Iteración {best_iteration} con score {best_score}")
    if improvement > 0:
        print(f"📈 Mejora total: {improvement:.6f}")
else:
    print("\n⚠️ No se pudo determinar el mejor modelo")

print(f"\n📁 Progreso guardado en: iteration_progress_{datetime.now().strftime('%Y%m%d')}.json")
